<a href="https://colab.research.google.com/github/MRifdaM/SurabayaDev_Asa-Baswara_Data/blob/main/Analisis_Penjualan_HP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning Data

In [1]:
# 0) IMPORT DAN MOUNT DRIVE
from google.colab import drive
import pandas as pd
import numpy as np
import re

drive.mount('/content/drive')

# 1) LOAD & NORMALISASI NAMA KOLOM
src_path = "/content/drive/MyDrive/SurabayaDev/penjualan_hp.csv" # Pastikan path ini benar
df = pd.read_csv(src_path)

def normalize_colname(c):
    c = str(c).strip().lower()
    c = re.sub(r'[^0-9a-z]+', '_', c)
    c = re.sub(r'_+', '_', c).strip('_')
    return c

df.columns = [normalize_colname(c) for c in df.columns]
print("Shape Awal:", df.shape)
print("Nama Kolom:", list(df.columns))

# 2) PEMETAAN ALIAS KOLOM (FLEKSIBEL TERHADAP NAMA KOLOM)
ALIASES = {
    "order_id": ["order_id", "id_order", "no_order", "invoice_id"],
    "tanggal_pembelian": ["tanggal_pembelian", "tanggal", "tgl_beli", "order_date", "date"],
    "brand": ["merek", "brand", "merk"],
    "produk": ["produk", "product", "tipe", "seri", "model"],
    "harga_satuan": ["harga_satuan", "harga", "price", "unit_price", "harga_per_unit"],
    "jumlah": ["jumlah", "qty", "kuantitas", "quantity"],
    "total_harga": ["total_harga", "total", "total_amount", "amount", "subtotal"],
    "metode_pembayaran": ["metode_pembayaran", "payment_method", "cara_bayar", "metode_bayar"],
    "kota": ["kota", "kota_kabupaten", "city", "kota_pembeli"],
    "status_pengiriman": ["status_pengiriman", "shipping_status", "status_kirim", "status"],
    "rating": ["rating", "penilaian", "skor", "nilai"],
    "nama_pelanggan": ["nama_pelanggan", "customer_name", "nama", "pembeli"]
}

def pick_col(df, ops):
    for o in ops:
        if o in df.columns: return o
    return None

COL = {k: pick_col(df, v) for k, v in ALIASES.items()}

# 3) CASTING TIPE DATA (KONVERSI KE FORMAT YANG BENAR)
def to_float_safe(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    s = re.sub(r'[^\d,.\-]', '', s)
    if ',' in s and '.' in s:
        if s.rfind('.') < s.rfind(','): # Format Eropa: 1.234,56
            s = s.replace('.', '').replace(',', '.')
    elif ',' in s: # Format Indonesia: 1234,56
        s = s.replace(',', '.')
    try:
        return float(s)
    except (ValueError, TypeError):
        return np.nan

for key in ["harga_satuan", "total_harga", "rating"]:
    c = COL.get(key)
    if c: df[c] = df[c].apply(to_float_safe)

if COL.get("jumlah"):
    df[COL["jumlah"]] = pd.to_numeric(df[COL["jumlah"]], errors='coerce').astype('Int64')

if COL.get("tanggal_pembelian"):
    df[COL["tanggal_pembelian"]] = pd.to_datetime(df[COL["tanggal_pembelian"]], errors='coerce')

# 4) STANDARDISASI NILAI KATEGORIKAL (DENGAN PENANGANAN NaN YANG BENAR)
def standardize_text_column(series, mapping=None, case='lower'):
    # Menggunakan np.where untuk menjaga NaN tetap NaN selama operasi string
    s_cleaned = pd.Series(np.where(series.isna(), np.nan, series.astype(str).str.strip()), index=series.index)

    if case == 'lower':
        s_cleaned = s_cleaned.str.lower()
    elif case == 'title':
        s_cleaned = s_cleaned.str.title()
    elif case == 'upper':
        s_cleaned = s_cleaned.str.upper()

    # Hapus teks "Nan" yang mungkin muncul dari konversi NaN ke string
    s_cleaned = s_cleaned.replace('Nan', np.nan)

    if mapping:
        s_cleaned = s_cleaned.replace(mapping)

    return s_cleaned

# Terapkan standardisasi
if COL.get("metode_pembayaran"):
    map_payment = {'cash on delivery': 'cod', 'bayar ditempat': 'cod', 'bayar di tempat': 'cod','transfer bank': 'transfer', 'tf': 'transfer', 'bank transfer': 'transfer','credit card': 'kartukredit', 'kartu kredit': 'kartukredit','ovo': 'e-wallet', 'gopay': 'e-wallet', 'dana': 'e-wallet', 'shopeepay': 'e-wallet'}
    df[COL["metode_pembayaran"]] = standardize_text_column(df[COL["metode_pembayaran"]], map_payment)

if COL.get("status_pengiriman"):
    map_status = {'delivered': 'terkirim', 'success': 'terkirim', 'complete': 'terkirim', 'sukses': 'terkirim','failed': 'gagal', 'cancelled': 'gagal', 'dibatalkan': 'gagal', 'returned': 'gagal','diproses': 'proses', 'processed': 'proses', 'dikirim': 'proses', 'on delivery': 'proses', 'shipping': 'proses'}
    df[COL["status_pengiriman"]] = standardize_text_column(df[COL["status_pengiriman"]], map_status)

if COL.get("brand"):
    df[COL["brand"]] = standardize_text_column(df[COL["brand"]], case='title')

if COL.get("produk"):
    df[COL["produk"]] = standardize_text_column(df[COL["produk"]], case='upper')

if COL.get("kota"):
    s_kota = standardize_text_column(df[COL["kota"]], case='title')
    df[COL["kota"]] = s_kota.str.replace(r'^(Kota|Kab\.?)\s+', '', regex=True)

# 5) IMPUTASI NILAI HILANG (TANPA FLAG)
def impute_num_by_group(df, col, by):
    med_global = df[col].median()
    if by and by in df.columns:
        med_by = df.groupby(by)[col].transform('median')
        df[col] = df[col].fillna(med_by)
    df[col] = df[col].fillna(med_global) # Isi sisa NaN dengan median global
    return df

def impute_cat_by_group(df, col, by, fallback="Unknown"):
    # Hitung mode global sebagai fallback utama
    mode_global = df[col].mode(dropna=True)
    mode_global = mode_global.iloc[0] if not mode_global.empty else fallback

    if by and by in df.columns:
        # Buat peta dari grup 'by' ke nilai mode 'col'
        mode_map = df[df[col].notna()].groupby(by)[col].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
        # Gunakan peta untuk mengisi NaN
        df[col] = df[col].fillna(df[by].map(mode_map))

    # Isi sisa NaN (jika grupnya tidak punya mode atau 'by' tidak ada) dengan mode global
    df[col] = df[col].fillna(mode_global)
    return df

brand_col = COL.get("brand")

# a. Imputasi Brand terlebih dahulu (kunci untuk imputasi lain)
if brand_col:
    df = impute_cat_by_group(df, brand_col, by=None, fallback="Unknown")

# b. Imputasi kolom lain menggunakan brand_col yang sudah bersih
for k in ["metode_pembayaran", "status_pengiriman", "kota", "produk"]:
    c = COL.get(k)
    if c: df = impute_cat_by_group(df, c, brand_col)

for k in ["harga_satuan", "total_harga", "jumlah", "rating"]:
    c = COL.get(k)
    if c: df = impute_num_by_group(df, c, brand_col)

# c. Imputasi Tanggal
if COL.get("tanggal_pembelian"):
    c = COL["tanggal_pembelian"]
    if df[c].notna().any():
        mode_date = df[c].mode(dropna=True).iloc[0] if not df[c].mode(dropna=True).empty else pd.Timestamp.now().normalize()
        df[c] = df[c].fillna(mode_date)

print("\nJumlah NaN setelah imputasi:")
print(df.isnull().sum())

# 6) KOREKSI LOGIKA HARGA, JUMLAH, DAN TOTAL
price_col = COL.get("harga_satuan")
qty_col   = COL.get("jumlah")
total_col = COL.get("total_harga")
assert price_col and qty_col and total_col, "Pastikan kolom harga_satuan, jumlah, total_harga ada."

# pastikan numerik
df[price_col] = pd.to_numeric(df[price_col], errors='coerce')
df[total_col] = pd.to_numeric(df[total_col], errors='coerce')
df[qty_col]   = pd.to_numeric(df[qty_col], errors='coerce').astype('Int64')

TOL = 0.02  # toleransi deviasi 2%

# ---- harga_satuan: imputasi dari total/qty
can_price = df[price_col].isna() & df[qty_col].notna() & (df[qty_col] > 0) & df[total_col].notna()
df.loc[can_price, price_col] = (df.loc[can_price, total_col] / df.loc[can_price, qty_col]).round(0)

# harga teoritis untuk koreksi & outlier
has_both = df[qty_col].notna() & (df[qty_col] > 0) & df[total_col].notna()
harga_teoritis = pd.Series(np.nan, index=df.index)
harga_teoritis.loc[has_both] = (df.loc[has_both, total_col] / df.loc[has_both, qty_col])

implausible_price = df[price_col].isin([1, 9_999_999, 99_999_999, 5_000_000_000, 5000000000]) | (df[price_col] <= 0)
deviasi_besar = (df[price_col].notna() & harga_teoritis.notna() &
                 ((df[price_col] - harga_teoritis).abs() / harga_teoritis.abs() > TOL))

fix_price_mask = (implausible_price | deviasi_besar) & harga_teoritis.notna()
df.loc[fix_price_mask, price_col] = harga_teoritis.loc[fix_price_mask].round(0)

# ---- jumlah: imputasi dari total/harga
qty_was_na_or_bad = df[qty_col].isna() | (df[qty_col] <= 0)
can_qty = qty_was_na_or_bad & df[price_col].notna() & (df[price_col] > 0) & df[total_col].notna()

qty_calc = (df.loc[can_qty, total_col] / df.loc[can_qty, price_col]).round().astype('Int64')
qty_calc = qty_calc.clip(lower=1)

df.loc[can_qty, qty_col] = qty_calc

# pastikan tipe Int64
df[qty_col] = df[qty_col].astype('Int64')

# 7) CAPPING OUTLIER (OPSIONAL, MEMBUAT KOLOM BARU)
def winsorize(series, k=1.5):
    q1, q3 = series.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower_bound, upper_bound = q1 - k*iqr, q3 + k*iqr
    return series.clip(lower=lower_bound, upper=upper_bound)

if price_col: df['harga_satuan_capped'] = df.groupby(brand_col)[price_col].transform(winsorize)
if total_col: df['total_harga_capped'] = df.groupby(brand_col)[total_col].transform(winsorize)
if qty_col:   df['jumlah_capped'] = df.groupby(brand_col)[qty_col].transform(winsorize)


# 8) DROP BARIS DENGAN NAMA PELANGGAN KOSONG
name_col = COL.get("nama_pelanggan")
if name_col:
    mask_empty_name = df[name_col].isna() | df[name_col].astype(str).str.strip().eq('')
    before = len(df)
    df = df.loc[~mask_empty_name].copy()
    print(f"\nDrop baris nama_pelanggan kosong: {before - len(df)} baris dihapus.")

# ==== IMPUTASI order_id: isi kosong dengan nilai sebelumnya + 1 (preserve prefix & zero-pad) ====
order_col = COL.get("order_id")
if not order_col:
    raise KeyError("Kolom 'order_id' tidak ditemukan. Periksa ALIASES untuk order_id.")

# Helper: parse prefix & angka dari sebuah ID, mis. 'ORD00001' -> ('ORD', 1, width=5)
_id_pat = re.compile(r'^([^\d]*)(\d+)$')  # prefix = non-digit (boleh kosong), num = digit (>=1)

def parse_id(x: str):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return (None, None, None)
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return (None, None, None)
    m = _id_pat.match(s)
    if not m:
        return (None, None, None)
    pfx, num_str = m.group(1), m.group(2)
    try:
        num = int(num_str)
    except:
        return (None, None, None)
    return (pfx, num, len(num_str))

def fmt_id(pfx: str, num: int, width: int):
    pfx = "" if pfx is None else pfx
    if width is None or width <= 0:
        return f"{pfx}{num}"
    return f"{pfx}{str(num).zfill(width)}"

# Normalisasi nilai kosong jadi NaN (supaya gampang dideteksi)
ser = df[order_col].astype(str)
ser = ser.where(~ser.str.strip().eq(''), np.nan)
ser = ser.where(~ser.str.lower().eq('nan'), np.nan)

# Iterasi baris demi baris mengikuti urutan df saat ini
last_pfx, last_num, last_width = None, None, None
out_vals = ser.copy()

for i in df.index:  # Iterate through the index of the filtered DataFrame
    val = out_vals.loc[i]  # Use .loc to access by index label
    if pd.isna(val):  # perlu imputasi
        if last_num is not None:  # ada nilai sebelumnya yang valid
            new_num = last_num + 1
            out_vals.loc[i] = fmt_id(last_pfx, new_num, last_width)
            last_num = new_num
        else:
            # kasus khusus: baris pertama kosong & belum ada "sebelumnya"
            last_pfx, last_num, last_width = "ORD", 1, 5
            out_vals.loc[i] = fmt_id(last_pfx, last_num, last_width)
    else:
        # ada nilai -> update "sebelumnya" jika bisa diparse
        pfx, num, width = parse_id(val)
        if num is not None:
            last_pfx, last_num, last_width = pfx, num, width
        else:
            # kalau format tidak bisa diparse, reset sequence
            last_pfx, last_num, last_width = None, None, None

# Tulis balik ke DataFrame
df[order_col] = out_vals

# 9) FINAL SAVE
out_path = "/content/drive/MyDrive/SurabayaDev/penjualan_hp_clean.csv"
df.to_csv(out_path, index=False)
print(f"\n✅ Proses Selesai! Data bersih disimpan ke: {out_path} | Shape Akhir: {df.shape}")
print("\nCek final jumlah NaN:")
print(df.isnull().sum())
print("\nContoh 5 baris data bersih:")
display(df.head())

Mounted at /content/drive
Shape Awal: (1575, 12)
Nama Kolom: ['order_id', 'tanggal_pembelian', 'nama_pelanggan', 'kota', 'produk', 'merek', 'harga_satuan', 'jumlah', 'total_harga', 'metode_pembayaran', 'rating', 'status_pengiriman']

Jumlah NaN setelah imputasi:
order_id             62
tanggal_pembelian     0
nama_pelanggan       61
kota                  0
produk                0
merek                 0
harga_satuan          0
jumlah                0
total_harga           0
metode_pembayaran     0
rating                0
status_pengiriman     0
dtype: int64

Drop baris nama_pelanggan kosong: 61 baris dihapus.

✅ Proses Selesai! Data bersih disimpan ke: /content/drive/MyDrive/SurabayaDev/penjualan_hp_clean.csv | Shape Akhir: (1514, 15)

Cek final jumlah NaN:
order_id               0
tanggal_pembelian      0
nama_pelanggan         0
kota                   0
produk                 0
merek                  0
harga_satuan           0
jumlah                 0
total_harga            0
metode_

,order_id,tanggal_pembelian,nama_pelanggan,kota,produk,merek,harga_satuan,jumlah,total_harga,metode_pembayaran,rating,status_pengiriman,harga_satuan_capped,total_harga_capped,jumlah_capped
0,ORD00001,2025-02-28,"Tgk. Eli Narpati, S.IP",Gorontalo,IPHONE 13,Apple,9443827.0,1,9443827.0,transfer,4.0,terkirim,9443827.0,9443827.0,1
1,ORD00002,2025-03-10,Hendra Pratiwi,Salatiga,VIVO V29,Vivo,8682500.0,1,8682500.0,kartukredit,4.0,dalam pengiriman,8682500.0,8682500.0,1
2,ORD00003,2025-03-10,"Tami Iswahyudi, S.Kom",Tual,REALME C55,Realme,6183710.0,1,6183710.0,transfer,1.0,terkirim,6183710.0,6183710.0,1
3,ORD00004,2025-03-10,Rafid Saputra,Binjai,VIVO Y27,Vivo,13747665.0,1,13747665.0,cod,3.0,terkirim,13747665.0,13747665.0,1
5,ORD00006,2025-03-10,T. Mariadi Hidayat,Depok,VIVO Y27,Vivo,13747665.0,1,13747665.0,e-wallet,3.0,terkirim,13747665.0,13747665.0,1


In [3]:
# === Download langsung dari Colab ke komputer Anda ===
from google.colab import files
files.download(out_path)  # out_path = "/content/drive/MyDrive/SurabayaDev/penjualan_hp_clean.csv"


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Feature Engineering

In [2]:

from google.colab import drive
import pandas as pd, numpy as np, re, os

# 0) Mount Drive
if not os.path.ismount("/content/drive"):
    drive.mount("/content/drive")

# 1) Path file input/output di Drive
IN_PATH  = "/content/drive/MyDrive/SurabayaDev/penjualan_hp_clean.csv"   # <-- ganti kalau perlu
OUT_PATH = "/content/drive/MyDrive/SurabayaDev/penjualan_hp_transformed.csv"
OUT_SKU_MONTHLY   = "/content/drive/MyDrive/SurabayaDev/sku_monthly.csv"
OUT_BRAND_MONTHLY = "/content/drive/MyDrive/SurabayaDev/brand_monthly.csv"
OUT_SHIP_MONTHLY  = "/content/drive/MyDrive/SurabayaDev/ship_monthly.csv"

# 2) Load
df = pd.read_csv(IN_PATH)
print(f"✅ Loaded: {IN_PATH} | shape={df.shape}")

# 3) Normalisasi nama kolom
def normalize_colname(c):
    c = str(c).strip().lower()
    c = re.sub(r'[^0-9a-z]+','_', c)
    c = re.sub(r'_+','_', c).strip('_')
    return c
df.columns = [normalize_colname(c) for c in df.columns]

# 4) Pemetaan alias → COL
ALIASES = {
    "order_id": ["order_id","id_order","no_order","invoice_id"],
    "tanggal_pembelian": ["tanggal_pembelian","tanggal","tgl_beli","order_date","date"],
    "brand": ["merek","brand","merk"],
    "produk": ["produk","product","tipe","seri","model"],
    "harga_satuan": ["harga_satuan","harga","price","unit_price","harga_per_unit"],
    "jumlah": ["jumlah","qty","kuantitas","quantity"],
    "total_harga": ["total_harga","total","total_amount","amount","subtotal"],
    "metode_pembayaran": ["metode_pembayaran","payment_method","cara_bayar","metode_bayar"],
    "kota": ["kota","kota_kabupaten","city","kota_pembeli"],
    "status_pengiriman": ["status_pengiriman","shipping_status","status_kirim","status"],
    "rating": ["rating","penilaian","skor","nilai"],
}
def pick_col(df_, ops):
    for o in ops:
        if o in df_.columns: return o
    return None
COL = {k: pick_col(df, v) for k,v in ALIASES.items()}

# 5) Pastikan tipe kolom kunci
dt_col    = COL.get("tanggal_pembelian")
price_col = COL.get("harga_satuan")
qty_col   = COL.get("jumlah")
total_col = COL.get("total_harga")
brand_col = COL.get("brand")
prod_col  = COL.get("produk")
pay_col   = COL.get("metode_pembayaran")
city_col  = COL.get("kota")
ship_col  = COL.get("status_pengiriman")
rate_col  = COL.get("rating")

missing  = [name for name in ["tanggal_pembelian","harga_satuan","jumlah","total_harga","brand","produk"] if COL.get(name) is None]
if missing:
    raise KeyError(f"Kolom wajib belum terdeteksi: {missing}. Cek kembali header CSV.")

df[dt_col]    = pd.to_datetime(df[dt_col], errors="coerce")
df[price_col] = pd.to_numeric(df[price_col], errors="coerce")
df[total_col] = pd.to_numeric(df[total_col], errors="coerce")
df[qty_col]   = pd.to_numeric(df[qty_col], errors="coerce").astype("Int64")

# 6) SKU
df["sku"] = (df[brand_col].astype(str).str.strip() + " " + df[prod_col].astype(str).str.strip()).str.strip()

# 7) Fitur waktu
df["year"]    = df[dt_col].dt.year
df["month"]   = df[dt_col].dt.month
df["ym"]      = df[dt_col].dt.to_period("M").astype(str)   # 'YYYY-MM'
df["quarter"] = df[dt_col].dt.quarter
df["dow"]     = df[dt_col].dt.dayofweek                    # 0=Mon
df["bulan_pembelian"] = df[dt_col].dt.strftime("%B")
df["hari_pembelian"]  = df[dt_col].dt.strftime("%A")

# 8) Harga efektif & diskon implisit
TOL = 0.02
df["unit_price_effective"] = (df[total_col] / df[qty_col]).replace([np.inf,-np.inf], np.nan)
df["delta_unit_price"]     = df["unit_price_effective"] - df[price_col]
base = (df[price_col] * df[qty_col]).replace(0, np.nan)
rel_diff = (base - df[total_col]) / base
df["dapat_diskon"] = rel_diff.gt(0.01)   # >1% → diskon

# diskon relatif thd median SKU
sku_median = df.groupby("sku")["unit_price_effective"].transform("median")
df["discount_ratio_vs_sku_median"] = 1 - (df["unit_price_effective"] / sku_median)

# 9) Bucket harga (qcut; fallback cut)
basis_price = df["unit_price_effective"].where(df["unit_price_effective"].notna(), df[price_col])
try:
    df["harga_bucket"] = pd.qcut(basis_price, q=3, labels=["low","mid","high"])
except ValueError:
    bins = [-np.inf, basis_price.median(), basis_price.quantile(0.9), np.inf]
    df["harga_bucket"] = pd.cut(basis_price, bins=bins, labels=["low","mid","high"])

# (opsional) bucket per brand
try:
    df["harga_bucket_brand"] = (
        df.groupby(brand_col)["unit_price_effective"]
          .transform(lambda s: pd.qcut(s, q=3, labels=["low","mid","high"]))
    )
except Exception:
    df["harga_bucket_brand"] = df["harga_bucket"]

# 10) Payment safety level
if pay_col:
    safety_map = {"cod":3, "e-wallet":2, "kartukredit":2, "transfer":1}
    df["payment_safety_level"] = df[pay_col].map(safety_map).fillna(1).astype(int)

# 11) Flags kualitas rating vs status
if rate_col:
    df["rating_high"] = (pd.to_numeric(df[rate_col], errors="coerce") >= 4).astype(int)
if rate_col and ship_col:
    df["rating_v_status_mismatch"] = ((df[ship_col] == "gagal") &
                                      (pd.to_numeric(df[rate_col], errors="coerce") >= 4)).astype(int)

# 12) Agregat cache
# a) per SKU bulanan
sku_monthly = (df.groupby(["ym","sku"], as_index=False)
                 .agg(qty=(qty_col,"sum"), revenue=(total_col,"sum")))

# b) per brand bulanan
brand_monthly = (df.groupby(["ym",brand_col], as_index=False)
                   .agg(qty=(qty_col,"sum"), revenue=(total_col,"sum")))

# c) status kirim rate per bulan (PERBAIKAN: tanpa reset_index duplikat kolom)
if ship_col:
    ship_counts = (df.groupby(['ym', ship_col], as_index=False)
                     .size()
                     .rename(columns={'size': 'count'}))
    ship_counts['rate'] = (ship_counts['count'] /
                           ship_counts.groupby('ym')['count'].transform('sum')).round(4)
    ship_monthly = ship_counts[['ym', ship_col, 'rate']]
else:
    ship_monthly = pd.DataFrame()

# 13) Kota utilitas (Top-10)
if city_col:
    top_cities = set(df[city_col].value_counts().head(10).index)
    df["is_top_city"] = df[city_col].apply(lambda x: "Top" if x in top_cities else "Other")

# 14) Save ke Drive
df.to_csv(OUT_PATH, index=False)
sku_monthly.to_csv(OUT_SKU_MONTHLY, index=False)
brand_monthly.to_csv(OUT_BRAND_MONTHLY, index=False)
if not ship_monthly.empty:
    ship_monthly.to_csv(OUT_SHIP_MONTHLY, index=False)

print("✅ Transform selesai & tersimpan:")
print(" -", OUT_PATH)
print(" -", OUT_SKU_MONTHLY)
print(" -", OUT_BRAND_MONTHLY)
if not ship_monthly.empty:
    print(" -", OUT_SHIP_MONTHLY)

# 15) Preview
print("\nPreview kolom kunci:")
show_cols = [dt_col, "ym", brand_col, prod_col, "sku",
             price_col, "unit_price_effective", "harga_bucket",
             qty_col, total_col, "dapat_diskon"]
show_cols = [c for c in show_cols if c in df.columns]
display(df[show_cols].head(10))


✅ Loaded: /content/drive/MyDrive/SurabayaDev/penjualan_hp_clean.csv | shape=(1514, 15)
✅ Transform selesai & tersimpan:
 - /content/drive/MyDrive/SurabayaDev/penjualan_hp_transformed.csv
 - /content/drive/MyDrive/SurabayaDev/sku_monthly.csv
 - /content/drive/MyDrive/SurabayaDev/brand_monthly.csv
 - /content/drive/MyDrive/SurabayaDev/ship_monthly.csv

Preview kolom kunci:


,tanggal_pembelian,ym,merek,produk,sku,harga_satuan,unit_price_effective,harga_bucket,jumlah,total_harga,dapat_diskon
0,2025-02-28,2025-02,Apple,IPHONE 13,Apple IPHONE 13,9443827.0,9443827.0,mid,1,9443827.0,False
1,2025-03-10,2025-03,Vivo,VIVO V29,Vivo VIVO V29,8682500.0,8682500.0,low,1,8682500.0,False
2,2025-03-10,2025-03,Realme,REALME C55,Realme REALME C55,6183710.0,6183710.0,low,1,6183710.0,False
3,2025-03-10,2025-03,Vivo,VIVO Y27,Vivo VIVO Y27,13747665.0,13747665.0,mid,1,13747665.0,False
4,2025-03-10,2025-03,Vivo,VIVO Y27,Vivo VIVO Y27,13747665.0,13747665.0,mid,1,13747665.0,False
5,2025-03-10,2025-03,Apple,IPHONE 14 PRO,Apple IPHONE 14 PRO,9443827.0,9443827.0,mid,1,9443827.0,False
6,2025-03-10,2025-03,Samsung,GALAXY S23,Samsung GALAXY S23,5131304.0,5131304.0,low,1,5131304.0,False
7,2025-07-01,2025-07,Apple,OPPO RENO10,Apple OPPO RENO10,16139773.0,16139773.0,high,2,32279546.0,False
8,2025-03-10,2025-03,Samsung,GALAXY A14,Samsung GALAXY A14,15965299.0,15965299.0,high,2,31930598.0,False
9,2025-03-10,2025-03,Oppo,OPPO RENO10,Oppo OPPO RENO10,16139773.0,16139773.0,high,2,32279546.0,False


In [4]:
# === Download langsung dari Colab ke komputer (cara A) ===
from google.colab import files
import os

# daftar file yang ingin diunduh
to_download = [OUT_PATH, OUT_SKU_MONTHLY, OUT_BRAND_MONTHLY]
if not ship_monthly.empty:
    to_download.append(OUT_SHIP_MONTHLY)

for p in to_download:
    if os.path.exists(p):
        print(f"Mengunduh: {p}")
        files.download(p)
    else:
        print(f"⚠️ File tidak ditemukan, lewati: {p}")


Mengunduh: /content/drive/MyDrive/SurabayaDev/penjualan_hp_transformed.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Mengunduh: /content/drive/MyDrive/SurabayaDev/sku_monthly.csv


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Mengunduh: /content/drive/MyDrive/SurabayaDev/ship_monthly.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>